In [82]:
%load_ext autoreload
%autoreload 2

import os, sys
import copy
import socket
from tqdm import tqdm
import torch
import pickle
from torch import optim
from torch.utils.tensorboard import SummaryWriter

#!pip install networkx matplotlib
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt

import sys, os
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "../")))
from libs.topology_manager import *
from libs import fl, nn, data, log

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [83]:
# Save Logs To File (info | debug | warning | error | critical) [optional]
log.init("info")
#log.init("info", "federated.log")
#log.init("debug", "flkafka.log")

In [84]:
adj_mat = np.array([[0,1,1,1,1,1],
             [1,0,0,0,0,0],
             [1,0,0,0,0,0],
             [1,0,0,0,0,0],
             [1,0,0,0,0,0], 
             [1,0,0,0,0,0]])

node_types = {'aggregator': [0,1], 'trainer': [2,3], 'broadcaster': [4,5]}

di_graph = nx.DiGraph(adj_mat)
#nx.draw(di_graph)

In [85]:
class FedArgs():
    def __init__(self):
        self.num_clients = len(adj_mat)
        self.epochs = 10
        self.local_rounds = 1
        self.client_batch_size = 32
        self.test_batch_size = 128
        self.learning_rate = 1e-4
        self.weight_decay = 1e-5
        self.cuda = False
        self.seed = 1
        self.topic = 'pyflx'
        self.tb = SummaryWriter('../out/runs/flkafka', comment="Mnist Distributed Federated training")

fedargs = FedArgs()

In [86]:
use_cuda = fedargs.cuda and torch.cuda.is_available()
torch.manual_seed(fedargs.seed)
device = torch.device("cuda" if use_cuda else "cpu")
kwargs = {"num_workers": 1, "pin_memory": True} if use_cuda else {}

In [87]:
host = socket.gethostname()
clients = [host + "(" + str(client + 1) + ")" for client in range(fedargs.num_clients)]
ctp = DistributedTopology(adj_mat, clients, node_types)

In [88]:
# Load MNIST Data to clients
train_data, test_data = data.load_dataset("mnist")
clients_data = data.split_data(train_data, clients)
client_train_loaders, client_test_loaders = data.load_client_data(clients_data, fedargs.client_batch_size, 0.2, **kwargs)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=fedargs.test_batch_size, shuffle=True, **kwargs)

clients_info = {
        client: {"train_loader": client_train_loaders[client],
                 "test_loader": client_test_loaders[client]}
        for client in clients
    }

In [89]:
import asyncio
import time

def background(f):
    def wrapped(*args, **kwargs):
        return asyncio.get_event_loop().run_in_executor(None, f, *args, **kwargs)

    return wrapped

@background
def process(client, epoch, ctp, model, train_loader, test_loader, fedargs, device):
    log.info("Processing Client {}".format(client))
    
    # Consume and Average, epoch passed is actually prev epoch, for which we want to consume updates
    rcvd_models = ctp.consume_model(client, fedargs.topic, model, epoch - 1)
    log.info("Client {} received {} model updates from {}".format(client, len(rcvd_models), list(rcvd_models.keys())))
    if len(rcvd_models) != 0:
        model = fl.federated_avg(rcvd_models)

    # Train    
    model, loss = fl.client_update(model,
                                train_loader,
                                fedargs.learning_rate,
                                fedargs.weight_decay,
                                fedargs.local_rounds,
                                device)
    
    # Publish
    ctp.produce_model(client, fedargs.topic, model, epoch)

    # Plot and Log
    for local_epoch, loss in enumerate(list(loss.values())):
        fedargs.tb.add_scalars("Training Loss/" + client, {str(epoch): loss}, str(local_epoch + 1))

    log.jsondebug(loss, "Epoch {} of {} : Federated Training loss, Client {}".format(epoch, fedargs.epochs, client))
    log.modeldebug(model, "Epoch {} of {} : Client {} Update".format(epoch, fedargs.epochs, client))

    # Test
    test_output = fl.eval(model, test_loader, device)
    fedargs.tb.add_scalar("Accuracy/" + client, test_output["accuracy"], epoch)
    log.jsoninfo(test_output, "Test Outut after Epoch {} of {} for Client {}".format(epoch, fedargs.epochs, client))

In [90]:
import time
start_time = time.time()

# Initialize Random Model
model = nn.ModelMNIST()

# Federated Training
for _epoch in tqdm(range(fedargs.epochs)):

    epoch = _epoch + 1
    log.info("Federated Training Epoch {} of {}".format(epoch, fedargs.epochs))

    # Clients  
    tasks = [process(client, epoch, ctp, model,
                     clients_info[client]['train_loader'],
                     clients_info[client]['test_loader'],
                     fedargs, device) for client in clients]
    await asyncio.wait(tasks)
    
print(time.time() - start_time)

  0%|          | 0/10 [00:00<?, ?it/s]2021-08-02 12:08:59,635 - <ipython-input-90-5572d9046366>::<module>(l:11) : Federated Training Epoch 1 of 10 [MainProcess : MainThread (INFO)]
2021-08-02 12:08:59,660 - <ipython-input-89-ea74ff6f3f6d>::process(l:12) : Processing Client bladecluster.iitp.org(4) [MainProcess : asyncio_4 (INFO)]
2021-08-02 12:08:59,710 - <ipython-input-89-ea74ff6f3f6d>::process(l:12) : Processing Client bladecluster.iitp.org(1) [MainProcess : asyncio_1 (INFO)]
2021-08-02 12:08:59,713 - <ipython-input-89-ea74ff6f3f6d>::process(l:12) : Processing Client bladecluster.iitp.org(2) [MainProcess : asyncio_2 (INFO)]
2021-08-02 12:08:59,713 - <ipython-input-89-ea74ff6f3f6d>::process(l:12) : Processing Client bladecluster.iitp.org(3) [MainProcess : asyncio_5 (INFO)]
2021-08-02 12:08:59,715 - <ipython-input-89-ea74ff6f3f6d>::process(l:12) : Processing Client bladecluster.iitp.org(6) [MainProcess : asyncio_0 (INFO)]
2021-08-02 12:08:59,719 - <ipython-input-89-ea74ff6f3f6d>::proce

 10%|█         | 1/10 [00:33<05:02, 33.58s/it]2021-08-02 12:09:33,217 - <ipython-input-90-5572d9046366>::<module>(l:11) : Federated Training Epoch 2 of 10 [MainProcess : MainThread (INFO)]
2021-08-02 12:09:33,283 - <ipython-input-89-ea74ff6f3f6d>::process(l:12) : Processing Client bladecluster.iitp.org(4) [MainProcess : asyncio_8 (INFO)]
2021-08-02 12:09:33,293 - <ipython-input-89-ea74ff6f3f6d>::process(l:12) : Processing Client bladecluster.iitp.org(3) [MainProcess : asyncio_11 (INFO)]
2021-08-02 12:09:33,294 - <ipython-input-89-ea74ff6f3f6d>::process(l:12) : Processing Client bladecluster.iitp.org(1) [MainProcess : asyncio_7 (INFO)]
2021-08-02 12:09:33,298 - <ipython-input-89-ea74ff6f3f6d>::process(l:12) : Processing Client bladecluster.iitp.org(6) [MainProcess : asyncio_6 (INFO)]
2021-08-02 12:09:33,299 - <ipython-input-89-ea74ff6f3f6d>::process(l:12) : Processing Client bladecluster.iitp.org(5) [MainProcess : asyncio_10 (INFO)]
2021-08-02 12:09:33,299 - <ipython-input-89-ea74ff6f3f

2021-08-02 12:10:07,102 - <ipython-input-89-ea74ff6f3f6d>::process(l:41) : Test Outut after Epoch 2 of 10 for Client bladecluster.iitp.org(1) {
    "accuracy": 91.9,
    "attack": {
        "attack_success_count": 0,
        "attack_success_rate": 0,
        "instances": 0,
        "misclassification_rate": 0,
        "misclassifications": 0
    },
    "correct": 1838,
    "test_loss": 0.27120948731899264
} [MainProcess : asyncio_7 (INFO)]
2021-08-02 12:10:07,547 - <ipython-input-89-ea74ff6f3f6d>::process(l:41) : Test Outut after Epoch 2 of 10 for Client bladecluster.iitp.org(5) {
    "accuracy": 93.0,
    "attack": {
        "attack_success_count": 0,
        "attack_success_rate": 0,
        "instances": 0,
        "misclassification_rate": 0,
        "misclassifications": 0
    },
    "correct": 1860,
    "test_loss": 0.2193467322587967
} [MainProcess : asyncio_10 (INFO)]
 20%|██        | 2/10 [01:07<04:32, 34.03s/it]2021-08-02 12:10:07,561 - <ipython-input-90-5572d9046366>::<module

2021-08-02 12:10:41,206 - /home/harsh_1921cs01/hub/F3IA/fl/libs/protobuf_producer.py::produce(l:56) : Producing user records to topic pyflx. ^C to exit. [MainProcess : asyncio_3 (INFO)]
2021-08-02 12:10:41,233 - <ipython-input-89-ea74ff6f3f6d>::process(l:41) : Test Outut after Epoch 3 of 10 for Client bladecluster.iitp.org(3) {
    "accuracy": 94.19999999999999,
    "attack": {
        "attack_success_count": 0,
        "attack_success_rate": 0,
        "instances": 0,
        "misclassification_rate": 0,
        "misclassifications": 0
    },
    "correct": 1884,
    "test_loss": 0.20737522596120833
} [MainProcess : asyncio_4 (INFO)]
2021-08-02 12:10:41,373 - /home/harsh_1921cs01/hub/F3IA/fl/libs/protobuf_producer.py::produce(l:66) : Flushing records... [MainProcess : asyncio_3 (INFO)]
2021-08-02 12:10:41,673 - <ipython-input-89-ea74ff6f3f6d>::process(l:41) : Test Outut after Epoch 3 of 10 for Client bladecluster.iitp.org(4) {
    "accuracy": 93.65,
    "attack": {
        "attack_suc

2021-08-02 12:11:15,510 - <ipython-input-89-ea74ff6f3f6d>::process(l:41) : Test Outut after Epoch 4 of 10 for Client bladecluster.iitp.org(2) {
    "accuracy": 94.45,
    "attack": {
        "attack_success_count": 0,
        "attack_success_rate": 0,
        "instances": 0,
        "misclassification_rate": 0,
        "misclassifications": 0
    },
    "correct": 1889,
    "test_loss": 0.18602037221193313
} [MainProcess : asyncio_6 (INFO)]
2021-08-02 12:11:15,803 - <ipython-input-89-ea74ff6f3f6d>::process(l:41) : Test Outut after Epoch 4 of 10 for Client bladecluster.iitp.org(1) {
    "accuracy": 95.25,
    "attack": {
        "attack_success_count": 0,
        "attack_success_rate": 0,
        "instances": 0,
        "misclassification_rate": 0,
        "misclassifications": 0
    },
    "correct": 1905,
    "test_loss": 0.16714750981330873
} [MainProcess : asyncio_9 (INFO)]
2021-08-02 12:11:15,815 - <ipython-input-89-ea74ff6f3f6d>::process(l:41) : Test Outut after Epoch 4 of 10 for 

...

2021-08-02 12:13:11,226 - <ipython-input-89-ea74ff6f3f6d>::process(l:16) : Client bladecluster.iitp.org(4) received 6 model updates from ['bladecluster.iitp.org(1)', 'bladecluster.iitp.org(2)', 'bladecluster.iitp.org(3)', 'bladecluster.iitp.org(6)', 'bladecluster.iitp.org(5)', 'bladecluster.iitp.org(4)'] [MainProcess : asyncio_8 (INFO)]
2021-08-02 12:13:11,241 - <ipython-input-89-ea74ff6f3f6d>::process(l:16) : Client bladecluster.iitp.org(2) received 6 model updates from ['bladecluster.iitp.org(1)', 'bladecluster.iitp.org(2)', 'bladecluster.iitp.org(3)', 'bladecluster.iitp.org(6)', 'bladecluster.iitp.org(5)', 'bladecluster.iitp.org(4)'] [MainProcess : asyncio_6 (INFO)]
2021-08-02 12:13:29,070 - /home/harsh_1921cs01/hub/F3IA/fl/libs/protobuf_producer.py::produce(l:56) : Producing user records to topic pyflx. ^C to exit. [MainProcess : asyncio_11 (INFO)]
2021-08-02 12:13:29,238 - /home/harsh_1921cs01/hub/F3IA/fl/libs/protobuf_producer.py::produce(l:66) : Flushing records... [MainProcess 

2021-08-02 12:13:46,543 - <ipython-input-89-ea74ff6f3f6d>::process(l:16) : Client bladecluster.iitp.org(2) received 6 model updates from ['bladecluster.iitp.org(3)', 'bladecluster.iitp.org(1)', 'bladecluster.iitp.org(2)', 'bladecluster.iitp.org(6)', 'bladecluster.iitp.org(4)', 'bladecluster.iitp.org(5)'] [MainProcess : asyncio_5 (INFO)]
2021-08-02 12:13:46,544 - <ipython-input-89-ea74ff6f3f6d>::process(l:16) : Client bladecluster.iitp.org(5) received 6 model updates from ['bladecluster.iitp.org(3)', 'bladecluster.iitp.org(1)', 'bladecluster.iitp.org(2)', 'bladecluster.iitp.org(6)', 'bladecluster.iitp.org(4)', 'bladecluster.iitp.org(5)'] [MainProcess : asyncio_3 (INFO)]
2021-08-02 12:13:46,545 - <ipython-input-89-ea74ff6f3f6d>::process(l:16) : Client bladecluster.iitp.org(4) received 6 model updates from ['bladecluster.iitp.org(3)', 'bladecluster.iitp.org(1)', 'bladecluster.iitp.org(2)', 'bladecluster.iitp.org(6)', 'bladecluster.iitp.org(4)', 'bladecluster.iitp.org(5)'] [MainProcess : a

2021-08-02 12:14:10,535 - <ipython-input-89-ea74ff6f3f6d>::process(l:12) : Processing Client bladecluster.iitp.org(3) [MainProcess : asyncio_7 (INFO)]
2021-08-02 12:14:10,562 - <ipython-input-89-ea74ff6f3f6d>::process(l:12) : Processing Client bladecluster.iitp.org(6) [MainProcess : asyncio_11 (INFO)]
2021-08-02 12:14:10,584 - <ipython-input-89-ea74ff6f3f6d>::process(l:12) : Processing Client bladecluster.iitp.org(5) [MainProcess : asyncio_10 (INFO)]
2021-08-02 12:14:10,585 - <ipython-input-89-ea74ff6f3f6d>::process(l:12) : Processing Client bladecluster.iitp.org(4) [MainProcess : asyncio_8 (INFO)]
2021-08-02 12:14:20,639 - <ipython-input-89-ea74ff6f3f6d>::process(l:16) : Client bladecluster.iitp.org(5) received 6 model updates from ['bladecluster.iitp.org(1)', 'bladecluster.iitp.org(6)', 'bladecluster.iitp.org(3)', 'bladecluster.iitp.org(4)', 'bladecluster.iitp.org(2)', 'bladecluster.iitp.org(5)'] [MainProcess : asyncio_10 (INFO)]
2021-08-02 12:14:20,640 - <ipython-input-89-ea74ff6f3f

2021-08-02 12:14:45,757 - <ipython-input-89-ea74ff6f3f6d>::process(l:41) : Test Outut after Epoch 10 of 10 for Client bladecluster.iitp.org(5) {
    "accuracy": 97.8,
    "attack": {
        "attack_success_count": 0,
        "attack_success_rate": 0,
        "instances": 0,
        "misclassification_rate": 0,
        "misclassifications": 0
    },
    "correct": 1956,
    "test_loss": 0.061430030670017004
} [MainProcess : asyncio_10 (INFO)]
100%|██████████| 10/10 [05:46<00:00, 34.61s/it]

346.15555453300476
